<a href="https://colab.research.google.com/github/srihitanuthalapati/BodyFatPrediction/blob/main/BodyFatPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"srihitanuthalapati","key":"fa29b797588fedd12e66720af19b668d"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download fedesoriano/body-fat-prediction-dataset

rm: cannot remove '/root/.kaggle': No such file or directory
  0% 0.00/7.43k [00:00<?, ?B/s]
100% 7.43k/7.43k [00:00<00:00, 13.1MB/s]


In [ ]:
! unzip body-fat-prediction-dataset

Archive:  body-fat-prediction-dataset.zip
  inflating: bodyfat.csv             


In [ ]:
data = pd.read_csv('bodyfat.csv')

In [ ]:
data.head()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(f"using device: {device}")

2.1.0+cu121
using device: cpu


In [ ]:

for col in data.columns:
  data[col] = data[col].astype('float32')
  if col != "Forearm" and col != "Density":
    data[col] = data[col].div(10)
data

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,1.23,2.3,15.425000,6.775,3.62,9.309999,8.52,9.450000,5.90,3.73,2.19,3.20,27.400000,1.71
1,1.0853,0.61,2.2,17.325001,7.225,3.85,9.360000,8.30,9.870000,5.87,3.73,2.34,3.05,28.900000,1.82
2,1.0414,2.53,2.2,15.400000,6.625,3.40,9.580000,8.79,9.920000,5.96,3.89,2.40,2.88,25.200001,1.66
3,1.0751,1.04,2.6,18.475000,7.225,3.74,10.180000,8.64,10.120000,6.01,3.73,2.28,3.24,29.400000,1.82
4,1.0340,2.87,2.4,18.424999,7.125,3.44,9.730000,10.00,10.190001,6.32,4.22,2.40,3.22,27.700001,1.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1.0736,1.10,7.0,13.425000,6.700,3.49,8.920000,8.36,8.880000,4.96,3.48,2.15,2.56,25.700001,1.85
248,1.0236,3.36,7.2,20.100000,6.975,4.09,10.850000,10.50,10.450000,5.96,4.08,2.32,3.52,28.600000,2.01
249,1.0328,2.93,7.2,18.674999,6.600,3.89,11.110000,11.15,10.170000,6.03,3.73,2.15,3.13,27.200001,1.80
250,1.0399,2.60,7.2,19.075001,7.050,3.89,10.830000,10.13,9.780001,5.60,4.16,2.27,3.05,29.400000,1.98


In [ ]:
class PredictionModel(nn.Module):
    def __init__(self):
        super(PredictionModel, self).__init__()
        input_size = 14
        hidden_size = 10
        output_size = 1
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
class BodyFatDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        a = self.data.iloc[index, 1:].values
        b = self.data.iloc[index, 1]
        return a, b

In [ ]:

model = PredictionModel()
model.to(device)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
body_dataset = BodyFatDataset(data)
batch_size=50
dataset_loader = torch.utils.data.DataLoader(dataset=body_dataset, batch_size=batch_size, shuffle=True)

epochs=300
input_size = 10
output_size = 1

for epoch in range(epochs):
  for a, b in dataset_loader:
    a, b = a.to(device), b.to(device)
    model.train()
    predictions = model(a)
    loss = criterion(predictions, b)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch [1/300], Loss: 1.9284
Epoch [1/300], Loss: 2.0161
Epoch [1/300], Loss: 2.0668
Epoch [1/300], Loss: 2.2210
Epoch [1/300], Loss: 2.0928
Epoch [1/300], Loss: 1.1101
Epoch [2/300], Loss: 2.0461
Epoch [2/300], Loss: 2.0435
Epoch [2/300], Loss: 2.0228
Epoch [2/300], Loss: 1.9102
Epoch [2/300], Loss: 1.9495
Epoch [2/300], Loss: 0.8633
Epoch [3/300], Loss: 1.8273
Epoch [3/300], Loss: 1.9248
Epoch [3/300], Loss: 1.9465
Epoch [3/300], Loss: 1.9775
Epoch [3/300], Loss: 1.9191
Epoch [3/300], Loss: 0.9649
Epoch [4/300], Loss: 1.9622
Epoch [4/300], Loss: 1.7985
Epoch [4/300], Loss: 1.6102
Epoch [4/300], Loss: 1.8801
Epoch [4/300], Loss: 1.9678
Epoch [4/300], Loss: 0.9133
Epoch [5/300], Loss: 1.8225
Epoch [5/300], Loss: 1.7872
Epoch [5/300], Loss: 1.7011
Epoch [5/300], Loss: 1.7389
Epoch [5/300], Loss: 1.7414
Epoch [5/300], Loss: 2.3512
Epoch [6/300], Loss: 1.7136
Epoch [6/300], Loss: 1.5351
Epoch [6/300], Loss: 1.6439
Epoch [6/300], Loss: 1.7178
Epoch [6/300], Loss: 1.8331
Epoch [6/300], Loss: